# Scalar advection-diffusion using sequential timestepping

We will solve scalar advection-diffusion using serial timestepping and the implicit theta method.

Scalar advection-diffusion is a model linear PDE describing the transport of a passive scalar $q$ by a specified velocity field $u$ and diffusion with a coefficient $\nu$:

$$
\partial_{t}q + u\partial_{x}q - \nu\partial^{2}_{xx}q = 0
$$

We will usually combine the two gradient terms into a single spatial residual matrix $\textbf{K}$:

$$
\partial_{t}q + \textbf{K}q = 0
$$


## Time discretisation

The implicit theta method is a single-step method which includes the first order forward and backward Euler methods and the second order trapezium rule method as special cases.

The solution $q$ is discretised in time, with the discrete solution $q^{n}$ approximating $q(t^{n})$, where $t^{n}=n\Delta t$ and $\Delta t$ is the timestep size. The implicit theta method provides an update from the current timestep $q^{n}$ to the next timestep $q^{n+1}$ by:

$$
\textbf{M}\frac{q^{n+1}-q^{n}}{\Delta t} + \theta \textbf{K}q^{n+1} + (1-\theta)\textbf{K}q^{n} = 0
$$

The time derivative is approximated using simple finite differences, and the right hand side is approximated by a weighted sum of the values at the current and next timestep. When $\theta=0$ we have the explicit forward Euler method, when $\theta=1$ we have the implicit backward Euler method, and when $\theta=0.5$ we have the trapezium rule. This method is first order in time unless $\theta=0.5$ for which it is second order.
We have included the mass matrix $\textbf{M}$ for completeness, but for finite difference and finite volume this is just the identity.

## Implementing the discretisation

Now that we have a timestepping method, we can begin implementing it.
We will solve the advection diffusion equation on a periodic 1D domain. The spatial gradient terms will be implemented using finite differences, so the domain is split into a number of discrete mesh points.
First we define the number of timesteps `nt`, the number of mesh points `nx`, the size of the domain `lx`.

The velocity is 1 everywhere, and the viscosity $\nu$ is calculated by enforcing the Reynolds number $Re=uL/\nu$, which describes whether advection or diffusion forces dominate. $Re\gg1$ will give an advection dominated flow which will not decay quickly.

The timestep is calculated by enforcing the CFL number $\sigma=u\Delta t/\Delta x$. If $\sigma<1$ then the advection velocity has travelled less than the distance between mesh points $\Delta x$ within one timestep, which will give higher accuracy than $\sigma>1$.

In [ ]:
from math import pi
nt = 256
nx = 128

lx = 2*pi
dx = lx/nx

theta = 0.5

# velocity, CFL, and reynolds number
u = 1
re = 500
cfl = 0.8

# calculate the viscosity and timestep
nu = lx*u/re
dt = cfl*dx/u

# advective and diffusive Courant numbers
cfl_u = cfl
cfl_v = nu*dt/dx**2

print(f"{nu = }, {dt = }, {cfl_v = }, {cfl_u = }")


The domain is $x\in[-l_{x}/2, l_{x}/2]$ and will be discretised by `nx` equally spaced grid points.

In [ ]:
import numpy as np

mesh = np.linspace(start=-lx/2, stop=lx/2, num=nx, endpoint=False)

The spatial derivative can be approximated by second order finite differences:

$$
\partial_{x}q|_{i} \approx \frac{q_{i+1}-q_{i-1}}{2\Delta x} + \mathcal{O}(\Delta x^{2})
$$
and for the second derivative:
$$
\partial^{2}_{xx}q|_{i} \approx \frac{q_{i+1}-2q_{i}+q_{i-1}}{\Delta x^{2}} + \mathcal{O}(\Delta x^{2}),
$$
where $q_{i}$ is the approximation of the solution at grid point $i$.

Two convenience functions are provided, `gradient_stencil` returns an array for a centred difference gradient stencil, and `sparse_circulant` returns a scipy sparse matrix for that stencil on a periodic mesh (which is itself circulant).

In [ ]:
from scipy import sparse

# Finite difference spatial discretisations                                                                                                                                                   
def gradient_stencil(grad, order):                                                                                                                                                            
    '''                                                                                                                                                                                       
    Return the centred stencil for the `grad`-th gradient                                                                                                                                     
    of order of accuracy `order`                                                                                                                                                              
    '''                                                                                                                                                                                       
    return {                                                                                                                                                                                  
        1: {  # first gradient                                                                                                                                                                
            2: np.array([-1/2, 0, 1/2]),                                                                                                                                                      
            4: np.array([1/12, -2/3, 0, 2/3, -1/12]),                                                                                                                                         
            6: np.array([-1/60, 3/20, -3/4, 0, 3/4, -3/20, 1/60])                                                                                                                             
        },                                                                                                                                                                                    
        2: {  # second gradient                                                                                                                                                               
            2: np.array([1, -2, 1]),                                                                                                                                                          
            4: np.array([-1/12, 4/3, -5/2, 4/3, -1/12]),                                                                                                                                      
            6: np.array([1/90, -3/20, 3/2, -49/18, 3/2, -3/20, 1/90])                                                                                                                         
        },                                                                                                                                                                                    
        4: {  # fourth gradient                                                                                                                                                               
            2: np.array([1,  -4, 6, -4, 1]),                                                                                                                                                  
            4: np.array([-1/6, 2, -13/2, 28/3, -13/2, 2, -1/6]),                                                                                                                              
            6: np.array([7/240, -2/5, 169/60, -122/15, 91/8, -122/15, 169/60, -2/5, 7/240])  # noqa: E501                                                                                     
        }                                                                                                                                                                                     
    }[grad][order]                                                                                                                                                                            
                                                                                                                                                                                              
                                                                                                                                                                                              
def sparse_circulant(stencil, n):                                                                                                                                                             
    '''                                                                                                                                                                                       
    Return sparse scipy matrix from finite difference                                                                                                                                         
    stencil on a periodic grid of size n.                                                                                                                                                     
    '''                                                                                                                                                                                       
    if len(stencil) == 1:                                                                                                                                                                     
        return sparse.spdiags([stencil[0]*np.ones(n)], 0)                                                                                                                                     
                                                                                                                                                                                              
    # extend stencil to include periodic overlaps                                                                                                                                             
    ns = len(stencil)                                                                                                                                                                         
    noff = (ns-1)//2                                                                                                                                                                          
    pstencil = np.zeros(ns+2*noff)                                                                                                                                                            
                                                                                                                                                                                              
    pstencil[noff:-noff] = stencil                                                                                                                                                            
    pstencil[:noff] = stencil[noff+1:]                                                                                                                                                        
    pstencil[-noff:] = stencil[:noff]                                                                                                                                                         
                                                                                                                                                                                              
    # constant diagonals of stencil entries                                                                                                                                                   
    pdiags = np.tile(pstencil[:, np.newaxis], n)                                                                                                                                              
                                                                                                                                                                                              
    # offsets for inner domain and periodic overlaps                                                                                                                                          
    offsets = np.zeros_like(pstencil, dtype=int)                                                                                                                                              
                                                                                                                                                                                              
    offsets[:noff] = [-n+1+i for i in range(noff)]                                                                                                                                            
    offsets[noff:-noff] = [-noff+i for i in range(2*noff+1)]                                                                                                                                  
    offsets[-noff:] = [n-noff+i for i in range(noff)]                                                                                                                                         
                                                                                                                                                                                              
    return sparse.spdiags(pdiags, offsets)

Now we can create the finite difference matrices for these operators. We also need a mass matrix for the time derivative, which for finite difference methods is just the identity matrix.

In [ ]:
# Mass matrix                                                                                                                                                                                 
M = sparse_circulant([1], nx)                                                                                                                                                                 
                                                                                                                                                                                              
# Advection matrix                                                                                                                                                                            
D = sparse_circulant(gradient_stencil(1, order=2), nx)                                                                                                                                        
                                                                                                                                                                                              
# Diffusion matrix                                                                                                                                                                            
L = sparse_circulant(gradient_stencil(2, order=2), nx)

Rearranging the implicit theta rule method, we can use $q^{n}$ to calculate $q^{n+1}$ by solving the implicit system:
$$
\left(\textbf{M} + \Delta t\theta \textbf{K}\right)q^{n+1}
= \left(\textbf{M} - \Delta t\left(1-\theta\right)\textbf{K}\right)q^{n}
$$
$$
\textbf{A}_{1}q^{n+1} = \textbf{A}_{0}q^{n}
$$

The matrices for the left and right hand sides are below. We attach a `solve` method to the left hand side by prefactoring the sparse matrix.

In [ ]:
from scipy.sparse import linalg as spla

# Spatial terms                                                                                                                                                                               
K = (u/dx)*D - (nu/dx**2)*L                                                                                                                                                                   
                                                                                                                                                                                              
# A0*q^{n} + A1*q^{n+1} = 0                                                                                                                                                                   
A0 = -M/dt + (1 - theta)*K                                                                                                                                                                    
A1 = M/dt + theta*K                                                                                                                                                                           
A1.solve = spla.factorized(A1.tocsc())

With the numerical scheme created, we can now set up the initial conditions (an isolated bump) and integrate N timesteps.

In [ ]:
# initial conditions                                                                                                                                                                          
qinit = np.zeros_like(mesh)                                                                                                                                                                   
qinit[:] = np.cos(mesh/2)**4                                                                                                                                                                  
                                                                                                                                                                                              
# calculate timeseries                                                                                                                                                                        
q = np.zeros((nt+1, len(qinit)))                                                                                                                                                              
q[0] = qinit                                                                                                                                                                                  
                                                                                                                                                                                              
for i in range(nt-1):                                                                                                                                                                         
    q[i+1] = A1.solve(-A0.dot(q[i]))

Visualise the solution with matplotlib. Only every `nplot` timesteps are plotted for clarity.

In [ ]:
import matplotlib.pyplot as plt                                                                                                                                                           
nplot = 32
plt.plot(mesh, qinit, label='ic')                                                                                                                                                         
for i in range(nplot, nt, nplot):                                                                                                                                       
    plt.plot(mesh, q[i+1], label=str(i))                                                                                                                                                  
plt.legend(loc='center left')                                                                                                                                                             
plt.grid()